In [ ]:
!pip -qq install rouge_score
!pip -qq install datasets
!pip -qq install spacy pytextrank
!python -m spacy download en_core_web_sm


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 110.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
from datasets import load_dataset
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import spacy
import pytextrank
import re
from tqdm import tqdm

# Loading spaCy and pyTextRank model
print("Loading spaCy and pyTextRank model...")
nlp = spacy.load("en_core_web_sm")
nlp.add_pipe("textrank")

# Preprocessing function
def preprocess_text(text):
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    return text

#  summarizing text using TextRank (using pytextrank)
def summarize_text_textrank(article, top_n=3):
    doc = nlp(article)
    summary = []
    for sent in doc._.textrank.summary(limit_sentences=top_n):
        summary.append(str(sent))
    return ' '.join(summary)

# Evaluation using ROUGE and BLEU with Smoothing
def evaluate_model_textrank(dataset, set_name):
    print(f"Evaluating {set_name} dataset using TextRank...")
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = {'rouge1': [], 'rouge2': [], 'rougeL': [], 'bleu': []}
    smooth = SmoothingFunction().method4  # Applying smoothing to BLEU

    # Using tqdm to show a progress bar during evaluation
    for item in tqdm(dataset):
        article = preprocess_text(item['Annual Reports'])
        gold_summary = preprocess_text(item['Gold Summaries'])

        if article:
            # Using TextRank for summarization
            generated_summary = summarize_text_textrank(article)

            # ROUGE scores
            score = scorer.score(gold_summary, generated_summary)
            scores['rouge1'].append(score['rouge1'].fmeasure)
            scores['rouge2'].append(score['rouge2'].fmeasure)
            scores['rougeL'].append(score['rougeL'].fmeasure)

            # BLEU score with smoothing
            reference = gold_summary.split()  # Tokenizing gold summary
            candidate = generated_summary.split()  # Tokenizing generated summary
            bleu_score = sentence_bleu([reference], candidate, smoothing_function=smooth)
            scores['bleu'].append(bleu_score)

    avg_rouge1 = sum(scores['rouge1']) / len(scores['rouge1'])
    avg_rouge2 = sum(scores['rouge2']) / len(scores['rouge2'])
    avg_rougeL = sum(scores['rougeL']) / len(scores['rougeL'])
    avg_bleu = sum(scores['bleu']) / len(scores['bleu'])

    return avg_rouge1, avg_rouge2, avg_rougeL, avg_bleu

# Implementing Main function
def main():
    print("Loading datasets...")
    validation_data = load_dataset("ragha92/FNS_Summarization", split="validation")
    test_data = load_dataset("ragha92/FNS_Summarization", split="test")

    # Increasing spaCy's max_length to handle large texts
    nlp.max_length = 5500000

    # Evaluation on validation set
    print("Starting evaluation on the validation set...")
    avg_rouge1, avg_rouge2, avg_rougeL, avg_bleu = evaluate_model_textrank(validation_data, "validation")
    print(f"Validation ROUGE-1: {avg_rouge1}, ROUGE-2: {avg_rouge2}, ROUGE-L: {avg_rougeL}, BLEU: {avg_bleu}")

    # Final evaluation on test set
    print("Starting evaluation on the test set...")
    avg_rouge1, avg_rouge2, avg_rougeL, avg_bleu = evaluate_model_textrank(test_data, "test")
    print(f"Test Set ROUGE-1: {avg_rouge1}")
    print(f"Test Set ROUGE-2: {avg_rouge2}")
    print(f"Test Set ROUGE-L: {avg_rougeL}")
    print(f"Test Set BLEU: {avg_bleu}")

# Defining Main function
if __name__ == "__main__":
    main()


/usr/local/lib/python3.10/dist-packages
Loading spaCy and pyTextRank model...
Loading datasets...


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/544M [00:00<?, ?B/s]

validation.csv:   0%|          | 0.00/68.6M [00:00<?, ?B/s]

...

Starting evaluation on the validation set...
Evaluating validation dataset using TextRank...


100%|██████████| 257/257 [43:21<00:00, 10.12s/it]


Validation ROUGE-1: 0.1932625670082543, ROUGE-2: 0.08460864879015939, ROUGE-L: 0.09984598900797564, BLEU: 0.04511367919983563
Starting evaluation on the test set...
Evaluating test dataset using TextRank...


100%|██████████| 259/259 [42:20<00:00,  9.81s/it]

Test Set ROUGE-1: 0.19802320544084856
Test Set ROUGE-2: 0.09089870229481341
Test Set ROUGE-L: 0.10519440059489231
Test Set BLEU: 0.04840078159345737
